# DEM - hole filling and ocean masking
A notebook filtering a DEM with a Gaussian filter for hole filling, and using a shapefile for defining the ocean. Plots are used to demonstrates several challenges with filtering in plots.

In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.interpolate

# Setup
### Parameters
sigma of 0.9 is minimum value to fill all holes

In [ ]:
sigma = 0.9
depth = -10

### Load in data
This is the DEM prepared in the _"Example_DEM_Shapefile_Filtering"_ notebook.

In [ ]:
# from Open Topography
dem=rioxarray.rioxarray.open_rasterio(r'C:\Users\pearsonra\Documents\data\herbert_10res\dem\dem_filled_and_fake_bathymetry.nc', masked=True)

### View and correct projections
#### Projection for DEM

In [ ]:
dem_tiff.rio.crs

### View DEM and select injection points
Zoom in on the head waters and select injection points.

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 5))
gs = f.add_gridspec(1, 3)

ax1 = f.add_subplot(gs[0, 0:2]) 
dem.plot(cmap="viridis", ax=ax1) # Greys_r

ax2 = f.add_subplot(gs[0,2])
dem.plot(cmap="viridis", ax=ax2, vmin = 0, vmax = 100)
matplotlib.pyplot.xlim([1.4237e6, 1.4285e6])

ax1.set_title("Filled DEM with faked Bathymetry")
ax2.set_title("Filled and Faked DEM Zoomed on head waters")

#### Use selection points slightly down-stream as DEM not ringed 

The DEM zoomed in on the region to manually create a polygon here

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 5))

ax1 = f.add_subplot(121) 
dem_tiff.plot(cmap="viridis", ax=ax1, vmin = 0, vmax = 100)
ax1.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4238e6, 1.425e6])
matplotlib.pyplot.ylim([4.9880e6, 4.9895e6])

ax2 = f.add_subplot(122) 
dem_tiff.plot(cmap="viridis", ax=ax2, vmin = 0, vmax = 100)
ax2.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4244e6, 1.4255e6])
matplotlib.pyplot.ylim([4.9835e6, 4.9855e6])

matplotlib.pyplot.show()

### Manually create a polygon

In [ ]:
injection_1 = shapely.geometry.Polygon([(1.42425e6, 4.9886e6), # x1, y1
                                               (1.42439e6, 4.9886e6), # x2, y1
                                               (1.42439e6, 4.98845e6), # x2, y2
                                               (1.42425e6, 4.98845e6)  # x1, y2
        ])
injection_1_gdf = geopandas.GeoSeries([injection_1])

injection_2 = shapely.geometry.Polygon([(1.42507e6, 4.9841e6), # x1, y1
                                        (1.42518e6, 4.9841e6), # x2, y1
                                        (1.42518e6, 4.98425e6), # x2, y2
                                        (1.42507e6, 4.98425e6)  # x1, y2
        ])
injection_2_gdf = geopandas.GeoSeries([injection_2])

#### Plot both together

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 5))

ax1 = f.add_subplot(121) 
dem_tiff.plot(cmap="viridis", ax=ax1, vmin = 0, vmax = 65)
ax1.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4238e6, 1.425e6])
matplotlib.pyplot.ylim([4.9880e6, 4.9895e6])
injection_1_gdf.plot(color='None', edgecolor='red', linewidth=2, ax=ax1, zorder=4)

ax2 = f.add_subplot(122) 
dem_tiff.plot(cmap="viridis", ax=ax2, vmin = 0, vmax = 65)
ax2.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4244e6, 1.4255e6])
matplotlib.pyplot.ylim([4.9835e6, 4.9855e6])
injection_2_gdf.plot(color='None', edgecolor='red', linewidth=2, ax=ax2, zorder=4)

matplotlib.pyplot.show()

# Save out in the right format 
## To BG-FLOOD parameter file
x,y pairs
## Create a water injection file